In [1]:
pip install bltk


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 17.4 MB 5.0 MB/s 
  Created wheel for bltk: filename=bltk-1.2-py3-none-any.whl size=17432538 sha256=b533b27707f744906a1264ced64ee7f618fc79f01492c3466002ead42037b19b
  Stored in directory: /root/.cache/pip/wheels/0b/4f/91/e074e661b4dcbc24a83e050d1c75cecfa186ffe9d58b641c51
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=671245cf6544833444e33d7aef94d63c4be7f93163998a9f3011ac9331cda74b
  Stored in directory: /root/.cache/pip/wheels/42/56/cc/4a8bf86613aafd5b7f1b310477667c1fca5c51c3ae4124a003
Successfully built bltk sklearn


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install bltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##**Importing Libraries**


In [4]:
from keras.layers.core import Dense, SpatialDropout1D
from keras.layers.convolutional import Conv1D
from tensorflow.keras.layers import Embedding
from keras.layers.pooling import GlobalMaxPooling1D
from keras.models import Sequential
from keras_preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from bltk.langtools import Tokenizer # BLTK: The Bengali Natural Language Processing Toolkit
from bltk.langtools import remove_stopwords
import collections
import nltk
import numpy as np
import codecs
from sklearn import metrics

In [5]:
from bltk.langtools.banglachars import (vowels,
                                        vowel_signs,
                                        consonants,
                                        digits,
                                        operators,
                                        punctuations,
                                        others)



In [6]:
print(f'Vowels: {vowels}')
print(f'Vowel signs: {vowel_signs}')
print(f'Consonants: {consonants}')
print(f'Digits: {digits}')
print(f'Operators: {operators}')
print(f'Punctuation marks: {punctuations}')
print(f'Others: {others}')

Vowels: ['অ', 'আ', 'ই', 'ঈ', 'উ', 'ঊ', 'ঋ', 'ঌ', 'এ', 'ঐ', 'ও', 'ঔ']
Vowel signs: ['া', 'ি', 'ী', 'ু', 'ূ', 'ৃ', 'ৄ', 'ে', 'ৈ', 'ো', 'ৌ']
Consonants: ['ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ', 'ধ', 'ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 'শ', 'ষ', 'স', 'হ', 'ড়', 'ঢ়', 'য়', 'ৎ', 'ং', 'ঃ', 'ঁ']
Digits: ['০', '১', '২', '৩', '৪', '৫', '৬', '৭', '৮', '৯']
Operators: ['=', '+', '-', '*', '/', '%', '<', '>', '×', '÷']
Punctuation marks: ['।', ',', ';', ':', '?', '!', "'", '.', '"', '-', '[', ']', '{', '}', '(', ')', '–', '—', '―', '~']
Others: ['৳', '৺', '্', 'ঀ', 'ঽ', '#', '$']


In [7]:
np.random.seed(42)
INPUT_FILE = "/content/drive/My Drive/499A_Project/Dataset/ecommerce_dataset.txt"
VOCAB_SIZE = 10000
EMBED_SIZE = 300
NUM_FILTERS = 256
NUM_WORDS = 3
BATCH_SIZE = 40
NUM_EPOCHS = 100


##**Dataset Preparation and Cleaning**

In [8]:
counter = collections.Counter()
tokenizer = Tokenizer()
maxlen = 0
xs_tfidf, ys_tfidf = [], []


fin = codecs.open(INPUT_FILE, "r", encoding='utf-16')
for line in fin:
    label, sent = line.strip().split("\t") #Stripping the dataset based on tab. That is stripping label from sentence
    print("Label: ", label)
    print("Sentence: ",sent)

    words = tokenizer.word_tokenizer(sent)
    print("Afert Tokenizing: ",words)

    wordsExcludingPunctuationMarks=[word for word in words if word not in punctuations]
    print("Truncating punctuation:", wordsExcludingPunctuationMarks)

    wordsExcludingStopWords=remove_stopwords(wordsExcludingPunctuationMarks, level='soft')
    print("Truncating StopWords:", wordsExcludingStopWords)

    if len(wordsExcludingStopWords) > maxlen: #For calculating the maximum number of words in a sentence
        maxlen = len(wordsExcludingStopWords) 
    for wordExcludingStopWords in wordsExcludingStopWords:
        counter[wordExcludingStopWords] += 1 #Putting the frequency of each  word in a dictionary
    print("***************************************************************************************")

    ys_tfidf.append(int(label))
    xs_tfidf.append(' '.join(wordsExcludingStopWords))

fin.close()

    


Streaming output truncated to the last 5000 lines.
Truncating StopWords: ['প্যাকেজিং', 'ভালো', 'প্রোডাক্ট', 'ও', 'ভালো', 'ওয়াটার', 'প্রুফ', 'বলতে', 'গেলে', 'তাড়াতাড়িই', 'প্রোডাক্ট', 'হাতে', 'পেয়েছি', 'সবমিলিয়ে', 'সন্তোষজনক']
***************************************************************************************
Label:  1
Sentence:  প্রোডাক্ট টা খুবই ভালো প্যাকেটিং টা অনেক সুন্দর ছিল ঘড়িটা অনেক সুন্দর এক কথায় বলতে গেলে অসাধারণ
Afert Tokenizing:  ['প্রোডাক্ট', 'টা', 'খুবই', 'ভালো', 'প্যাকেটিং', 'টা', 'অনেক', 'সুন্দর', 'ছিল', 'ঘড়িটা', 'অনেক', 'সুন্দর', 'এক', 'কথায়', 'বলতে', 'গেলে', 'অসাধারণ']
Truncating punctuation: ['প্রোডাক্ট', 'টা', 'খুবই', 'ভালো', 'প্যাকেটিং', 'টা', 'অনেক', 'সুন্দর', 'ছিল', 'ঘড়িটা', 'অনেক', 'সুন্দর', 'এক', 'কথায়', 'বলতে', 'গেলে', 'অসাধারণ']
Truncating StopWords: ['প্রোডাক্ট', 'টা', 'ভালো', 'প্যাকেটিং', 'টা', 'অনেক', 'সুন্দর', 'ঘড়িটা', 'অনেক', 'সুন্দর', 'এক', 'কথায়', 'বলতে', 'গেলে', 'অসাধারণ']
********************************************************************

In [9]:
print("Maximum number of word in a sentence: ",maxlen)
print("Frequency of each words: ")
counter

Maximum number of word in a sentence:  29
Frequency of each words: 


Counter({'অনেকগুলা': 1,
         'অরডার': 4,
         'একটু': 36,
         'দেখবেন': 2,
         'ভালোবাসা': 2,
         'রইল': 8,
         'ইভ্যালির': 4,
         'প্রতি': 6,
         'আগের': 7,
         'প্রডাক্ট': 51,
         'ক্লিয়ার': 1,
         'করেন': 15,
         'তারাতাড়ি': 2,
         'আর': 87,
         'ভাল': 101,
         'লাগতেছে': 2,
         'দয়া': 3,
         'বলেন': 6,
         'ভাই': 43,
         'কবে': 8,
         'পাবো': 14,
         'সঠিক': 10,
         'তারিখে': 12,
         'দিতেন': 1,
         'কেউ': 26,
         'অভিযোগ': 8,
         'দিত': 2,
         'ই': 23,
         'কমার্সের': 1,
         'নামে': 9,
         'আপনারা': 33,
         'সাধারণ': 3,
         'মানুষের': 8,
         'সাথে': 34,
         'যা': 10,
         'করতেছে': 3,
         'একদিন': 4,
         'হিসাব': 1,
         'আপনাদের': 101,
         'কড়ায়': 1,
         'ঘন্ডায়': 1,
         'দিতে': 18,
         'ফাইজলামি!!': 1,
         'দীর্ঘ': 2,
         'হায়াত': 1,
         'কামনা': 11,
         

##**Generating Word2index**

In [10]:
word2index = collections.defaultdict(int) 
for wid, word in enumerate(counter.most_common(VOCAB_SIZE)):
    word2index[word[0]] = wid + 1

word2index  

defaultdict(int,
            {'ভালো': 1,
             'অনেক': 2,
             'অর্ডার': 3,
             'প্রোডাক্ট': 4,
             'ডেলিভারি': 5,
             'সুন্দর': 6,
             'দাম': 7,
             'ভাল': 8,
             'আপনাদের': 9,
             'আর': 10,
             'টা': 11,
             'ধন্যবাদ': 12,
             '': 13,
             'নাই': 14,
             'ও': 15,
             'বেশি': 16,
             'পেয়েছি': 17,
             'টাকা': 18,
             'পণ্য': 19,
             'আলহামদুলিল্লাহ': 20,
             'প্রডাক্ট': 21,
             'হাতে': 22,
             'দিন': 23,
             'এটা': 24,
             'কোয়ালিটি': 25,
             'কম': 26,
             'মান': 27,
             'ভাই': 28,
             'বাজে': 29,
             'এক': 30,
             'নিতে': 31,
             'অসাধারণ': 32,
             'করা': 33,
             'সার্ভিস': 34,
             'খারাপ': 35,
             'গুলো': 36,
             'জিনিস': 37,
             'এখনো': 38,
             'পেলা

In [11]:
vocab_sz = len(word2index) + 1
print("Number of Vocabularies: ",vocab_sz)

Number of Vocabularies:  3425


In [12]:
xs, ys = [], []
fin = codecs.open(INPUT_FILE, "r", encoding='utf-16')
for line in fin:
    label, sent = line.strip().split("\t")
    
    words = tokenizer.word_tokenizer(sent)
    wordsExcludingPunctuationMarks=[word for word in words if word not in punctuations]
    wordsExcludingStopWords=remove_stopwords(wordsExcludingPunctuationMarks, level='soft')
    
    wids = [word2index[word] for word in wordsExcludingStopWords]

    ys.append(int(label))
    xs.append(wids)
fin.close()

In [13]:
xs

[[1226, 443, 43, 760],
 [761, 223, 444, 294],
 [255, 21, 1227, 114, 762],
 [10, 8, 763],
 [558, 43, 295, 28, 224, 122],
 [176, 145, 1228, 64, 225, 764],
 [73, 1229, 200, 48, 559, 226, 46, 177, 560, 445, 1230, 9, 1231, 1232, 95],
 [1233],
 [765, 1234, 158, 56],
 [28, 10, 201, 47, 102, 18, 14, 159, 1235, 561, 1236, 296],
 [28, 1237, 1238, 256, 95, 1239, 202, 446],
 [766, 1240, 447, 1241, 1242, 203],
 [57, 227, 103, 767, 448, 1243, 1244, 76, 56, 90, 1245, 12],
 [28, 41, 36, 224, 1246],
 [1247, 294, 1248, 204, 203, 110, 123, 3],
 [21, 36, 297, 367, 28],
 [146, 1249, 14, 298, 768, 5, 1250],
 [48, 10, 562, 226, 563, 1251, 52, 564, 1252],
 [1253, 1254, 1255, 57, 115, 96, 1256, 565, 299],
 [300, 1257, 160, 1258, 52, 564, 12],
 [1259, 566, 74, 1260, 1261, 13],
 [1, 1262, 1, 449, 448, 567],
 [205, 1263, 1264],
 [178, 1265, 1266, 1267, 447, 1268],
 [1269, 3, 36, 368, 450, 24, 43, 124, 568, 1270],
 [1271, 3, 52, 769, 569, 1272, 19, 5, 1273],
 [226, 770, 18, 1274, 1275],
 [1276, 1277, 206, 771, 127

##**TF-IDF**

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(xs_tfidf, ys_tfidf, test_size=0.25, random_state=0)
Xtrain

['খুবি বাজে কোয়ালিটি ভাই ডিজাইন ভালো লাগছিল কিনছিলাম এত দাম দিয়েও বাট পুরাই বাজে',
 '"আলহামদুলিল্লাহ ওর্ডার করছি সেটাই পাইছি।অল্প দামে ভালো প্রোডাক্ট',
 'মাসে ৫/৬ বার ডেলিভারি ম্যান আসতো প্রডাক্ট ডেলিভারি দিতে,,এখন এক এক বারও আসে না,,কারণ ডেলিভারি চার্জ বাড়ানোর অর্ডারই করি আর',
 'ই ভালো লেগেছে কাছে',
 'বলে এক দেয় আরেক ৷ চিটিং',
 'বেশ খারাপ অভিজ্ঞতা',
 'দুইটা নিয়েছি অনেক সুন্দর',
 'হাইলি রিকমেন্ডড',
 'রমজানের টি-শার্ট দাম ৪৯৫ টাকা ঈদের বেড়ে ৫৯৫ টাকা সবচেয়ে বড় কথা হল এক সপ্তার কালার চলে গেল',
 'অনেক অনেক ধন্যবাদ অনেক ভাল মানের কাপড়',
 'স্নোবল ফেয়ারি লাইটস টা সত্যিই অসম্ভব সুন্দর অনেক কোয়ালিটি ফুল লাইট',
 'ধোঁকা বাজি',
 'পণ্যের দাম বেশি দেখিয়ে তারপরে ডিসকাউন্ট দেয়া এটা শুভঙ্করের ফাঁকি',
 'মনে বছর আর পাব',
 'কোয়ালিটি এভারেজ সেলাইগুলো দুর্বল মনে তবে ব্যাগটা দেখতে সুন্দর বাচ্চাদের সাথে অনেক মানানসই ভারী বহন করাই বুদ্ধিমানের কাজ',
 'ওয়েবসাইট',
 'জিনিস কবে পাব',
 'খারাপ মানের খাবার',
 'এমন জঘন্য বার্গার সত্যি টাকাই লস',
 'দোকানের পরিষেবা নিয়ে অত্যন্ত সন্তুষ্ট',
 'আলহামদুলিল্লাহ প্রোডাক

In [16]:
#tf idf
tf_idf = TfidfVectorizer()
#applying tf idf to training data
Xtrain_tf = tf_idf.fit_transform(Xtrain)
#applying tf idf to training data
Xtrain_tf = tf_idf.transform(Xtrain)
print("n_samples: %d, n_features: %d" % Xtrain_tf.shape)

n_samples: 1223, n_features: 735


In [17]:
#transforming test data into tf-idf matrix
Xtest_tf = tf_idf.transform(Xtest)
print("n_samples: %d, n_features: %d" % Xtest_tf.shape)

n_samples: 408, n_features: 735


##**Training**


In [18]:
X = pad_sequences(xs, maxlen=maxlen)          #This code pads sequences and converts them to numpy arrays.
Y = np_utils.to_categorical(ys)                                              #This code is for preparing the data to be used in a neural network. 
                                              #The data is first padded so that all of the sequences are the same length. 
                                              #Then, the data is converted into a one-hot encoding(a type of encoding where each column corresponds to a single class).    
                     
                                                   


In [19]:
X

array([[   0,    0,    0, ...,  443,   43,  760],
       [   0,    0,    0, ...,  223,  444,  294],
       [   0,    0,    0, ..., 1227,  114,  762],
       ...,
       [   0,    0,    0, ...,    0, 3422, 3423],
       [   0,    0,    0, ...,    0,  557, 3424],
       [   0,    0,    0, ...,    0,  230, 1216]], dtype=int32)

In [20]:

Y


array([[1., 0.],
       [0., 1.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [21]:

Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.20, random_state=42)          #This code splits data into training and testing sets. 
Y = np_utils.to_categorical(ys)                                                                 #The training set is used to train the model while the testing set is used to test the model. 
print(Xtrain.shape, Xtest.shape, Ytrain.shape, Ytest.shape)                                                                                               #The size of the training set and testing set can be specified. 
                                                                                                #In this case, the training set is 80% of the data while the testing set is 20% of the data. 
                                                                                                #The random_state parameter is used to seed the random number generator.


(1304, 29) (327, 29) (1304, 2) (327, 2)


##**CNN**

In [22]:
model = Sequential()                                                              #The code creates a Sequential model. A Sequential model is a linear stack of layers.
model.add(Embedding(vocab_sz, EMBED_SIZE, input_length=maxlen))                   #This code is adding an embedding layer to the model.
                                                                                  # The columns are independent of each other and the class is represented by a 1 in the column for that class and 0s in all other columns.
model.add(SpatialDropout1D(0.3))                                                  # The code above adds a SpatialDropout1D layer to the model with a dropout rate of 0.3.
model.add(Conv1D(filters=NUM_FILTERS,kernel_size=NUM_WORDS,activation="relu"))    # This is a convolutional neural network layer with one dimensional data. 
                                                                                  #The number of filters is specified by the NUM_FILTERS variable and the size of the kernel is specified by the NUM_WORDS variable.
                                                                                  #The activation function used is the rectified linear unit.
model.add(GlobalMaxPooling1D())                                                   # This code is adding a global max pooling layer to the model.(it'staking the maximum value from each feature map.)
model.add(Dense(2, activation="softmax"))                                         #This code creates a dense layer with 2 output nodes and a softmax activation function.

In [23]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])  #This code is compiling the model using the "adam" optimizer,
                                                                                        # the "categorical_crossentropy" loss function, and the "accuracy" metric.

In [24]:
history = model.fit(Xtrain, Ytrain, batch_size=BATCH_SIZE,epochs=NUM_EPOCHS,validation_data=(Xtest, Ytest))
#The code is training a model on a dataset using the fit method.
#The model is being trained on the Xtrain and Ytrain data,
# and the batch size and number of epochs are being specified. The model is being validated on the Xtest and Ytest data.

Epoch 1/100
33/33 [==============================] - 13s 16ms/step - loss: 0.6369 - accuracy: 0.6342 - val_loss: 0.5287 - val_accuracy: 0.8073
Epoch 2/100
33/33 [==============================] - 0s 6ms/step - loss: 0.3491 - accuracy: 0.8957 - val_loss: 0.3604 - val_accuracy: 0.8563
Epoch 3/100
33/33 [==============================] - 0s 6ms/step - loss: 0.1200 - accuracy: 0.9632 - val_loss: 0.3511 - val_accuracy: 0.8746
Epoch 4/100
33/33 [==============================] - 0s 6ms/step - loss: 0.0328 - accuracy: 0.9954 - val_loss: 0.3669 - val_accuracy: 0.8777
Epoch 5/100
33/33 [==============================] - 0s 6ms/step - loss: 0.0111 - accuracy: 0.9985 - val_loss: 0.3956 - val_accuracy: 0.8685
Epoch 6/100
33/33 [==============================] - 0s 6ms/step - loss: 0.0056 - accuracy: 0.9992 - val_loss: 0.4135 - val_accuracy: 0.8654
Epoch 7/100
33/33 [==============================] - 0s 6ms/step - loss: 0.0033 - accuracy: 1.0000 - val_loss: 0.4340 - val_accuracy: 0.8654
Epoch 8/100

In [25]:
y_pred=np.argmax(model.predict(Xtest), axis=-1)
y_test=np.argmax(Ytest, axis=1)                  #The code is predicting the labels for the Xtest data and comparing it to the known labels stored in Ytest.
                                                 # The output is the accuracy of the model.

11/11 [==============================] - 0s 2ms/step


In [26]:
print("Accuracy: ",metrics.accuracy_score(y_test, y_pred))
print(metrics.confusion_matrix(y_test,y_pred))

Accuracy:  0.8715596330275229
[[133  24]
 [ 18 152]]
